# Find Sentinel 1 acquisitions that overlap with STK intervals

For convenience, the badge below launches this notebook on Google [Colab](https://colab.research.google.com/).

<a target="_blank" href="https://colab.research.google.com/github/c-core-labs/notebooks/blob/master/notebooks/sentinel-1-acquisitions.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Install dependencies

In [1]:
import sys
!{sys.executable} -m pip install pandas python-dateutil sentinelsat


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


## Import dependencies

In [2]:
from typing import Dict
import datetime
import pandas as pd
from dateutil.parser import parse
from sentinelsat import SentinelAPI, geojson_to_wkt

## SCIHUB credentials

In [3]:
scihub_username = ""
scihub_password = ""

## Path to STK output

In [4]:
path = "sentinel-1-acquisitions.txt"

## Convenience functions to extract start and stop datetimes and data types

In [5]:
def get_start(line: str):
    line_stripped = line.strip()
    start = parse("-".join(line_stripped.split()[1:5]))
    
    return start


def get_end(line: str):
    line_stripped = line.strip()
    start = parse("-".join(line_stripped.split()[5:9]))
    
    return start


def items_to_row(items):
    """Given a list or Sentinel 1 items, summarize which data types are available."""
    row = {
        "raw": False, 
        "grd": False,  
        "slc": False,  
        "ocn": False, 
        "raw_duration": None, 
        "grd_duration": None,
        "slc_duration": None,
        "ocn_duration": None,
        "raw_id": None,
        "grd_id": None,
        "slc_id": None,
        "ocn_id": None,
    }
    for item in items:
        data_type = item["producttype"].lower()
        duration = item["endposition"] - item["beginposition"]
        duration = duration.seconds
        id = item["identifier"]

        row[data_type] = True
        row[f"{data_type}_duration"] = duration
        row[f"{data_type}_id"] = id
        
        
    return row

## Search for Sentinel 1 acquisitions, filtering by millisecond precision
The default search API appears to miss acquisitions if datetime interval is less than a minute. 
This function buffers the datetime interval for the search API, and performs a final precise datetime filter in memory.

In [6]:
def search(
    datetime_start: datetime.datetime,
    datetime_end: datetime.datetime,
    platformname: str = "Sentinel-1",
    timedelta: int = 1,
    verbose: bool = False,
) -> Dict:
    """
    Search Sentinel Hub
    """
    api = SentinelAPI(scihub_username, scihub_password)

    datetime_start_buffered = datetime_start - datetime.timedelta(minutes=timedelta)
    datetime_end_buffered = datetime_end + datetime.timedelta(minutes=timedelta)
    
    if verbose:
        print("Precise datetime range", datetime_start, datetime_end)
        print("Buffered datetime range",datetime_start_buffered, datetime_end_buffered)
    
    results = api.query(
        date=(datetime_start_buffered, datetime_end_buffered),
        platformname=platformname
    )

    items = [item[1] for item in list(results.items()) if item[1]["beginposition"] <= datetime_start and item[1]["endposition"] >= datetime_start]
    row = items_to_row(items)
    
    if len(items):
        print(f"Sentinel 1 acquisition found for {datetime_start} {datetime_end}")
    else:
        print(f"Sentinel 1 acquisition not found for {datetime_start} {datetime_end}")
    
    
    return row


## Read in the STK output and parse date intervals

In [7]:
with open(path) as source:
    lines = source.readlines()
    
starts = []
ends = []

for line in lines[8:]:
    line_stripped = line.strip()
    if line_stripped and line_stripped[0].isdigit():
        start = get_start(line_stripped)
        end = get_end(line_stripped)
        starts.append(start)
        ends.append(end)
        
intervals = pd.DataFrame({"start": starts, "end": ends})
intervals

,start,end
0,2022-02-07 05:54:08.340,2022-02-07 05:54:08.457
1,2022-02-13 11:40:15.318,2022-02-13 11:40:15.387
2,2022-02-19 19:51:11.360,2022-02-19 19:51:11.405
3,2022-02-26 12:56:07.603,2022-02-26 12:56:07.660
4,2022-03-05 06:01:02.927,2022-03-05 06:01:02.980
5,2022-03-07 09:49:34.693,2022-03-07 09:49:34.755
6,2022-03-09 12:00:44.707,2022-03-09 12:00:44.772
7,2022-03-13 16:23:36.253,2022-03-13 16:23:36.321
8,2022-03-26 02:22:48.160,2022-03-26 02:22:48.456
9,2022-04-09 13:08:05.246,2022-04-09 13:08:05.397


## Search each interval for Sentinel 1 acquisitions

In [8]:
rows = []

for index, interval in intervals.iterrows():
    row = search(interval.start, interval.end, timedelta=1)
    rows.append(row)
    
acquisition_results = pd.DataFrame(rows)
acquisitions = pd.concat([intervals, acquisition_results], axis=1)
acquisitions

Sentinel 1 acquisition found for 2022-02-07 05:54:08.340000 2022-02-07 05:54:08.457000
Sentinel 1 acquisition not found for 2022-02-13 11:40:15.318000 2022-02-13 11:40:15.387000
Sentinel 1 acquisition not found for 2022-02-19 19:51:11.360000 2022-02-19 19:51:11.405000
Sentinel 1 acquisition not found for 2022-02-26 12:56:07.603000 2022-02-26 12:56:07.660000
Sentinel 1 acquisition found for 2022-03-05 06:01:02.927000 2022-03-05 06:01:02.980000
Sentinel 1 acquisition not found for 2022-03-07 09:49:34.693000 2022-03-07 09:49:34.755000
Sentinel 1 acquisition not found for 2022-03-09 12:00:44.707000 2022-03-09 12:00:44.772000
Sentinel 1 acquisition found for 2022-03-13 16:23:36.253000 2022-03-13 16:23:36.321000
Sentinel 1 acquisition not found for 2022-03-26 02:22:48.160000 2022-03-26 02:22:48.456000
Sentinel 1 acquisition found for 2022-04-09 13:08:05.246000 2022-04-09 13:08:05.397000
Sentinel 1 acquisition not found for 2022-04-24 02:52:36.911000 2022-04-24 02:52:36.960000
Sentinel 1 acqu

,start,end,raw,grd,slc,ocn,raw_duration,grd_duration,slc_duration,ocn_duration,raw_id,grd_id,slc_id,ocn_id
0,2022-02-07 05:54:08.340,2022-02-07 05:54:08.457,True,True,True,True,34.0,35.0,35.0,35.0,S1A_IW_RAW__0SDV_20220207T055356_20220207T0554...,S1A_IW_GRDH_1SDV_20220207T055356_20220207T0554...,S1A_IW_SLC__1SDV_20220207T055356_20220207T0554...,S1A_IW_OCN__2SDV_20220207T055356_20220207T0554...
1,2022-02-13 11:40:15.318,2022-02-13 11:40:15.387,False,False,False,False,NaN,NaN,NaN,NaN,None,None,None,None
2,2022-02-19 19:51:11.360,2022-02-19 19:51:11.405,False,False,False,False,NaN,NaN,NaN,NaN,None,None,None,None
3,2022-02-26 12:56:07.603,2022-02-26 12:56:07.660,False,False,False,False,NaN,NaN,NaN,NaN,None,None,None,None
4,2022-03-05 06:01:02.927,2022-03-05 06:01:02.980,True,True,True,False,32.0,24.0,26.0,NaN,S1A_IW_RAW__0SDV_20220305T060101_20220305T0601...,S1A_IW_GRDH_1SDV_20220305T060040_20220305T0601...,S1A_IW_SLC__1SDV_20220305T060039_20220305T0601...,None
5,2022-03-07 09:49:34.693,2022-03-07 09:49:34.755,False,False,False,False,NaN,NaN,NaN,NaN,None,None,None,None
6,2022-03-09 12:00:44.707,2022-03-09 12:00:44.772,False,False,False,False,NaN,NaN,NaN,NaN,None,None,None,None
7,2022-03-13 16:23:36.253,2022-03-13 16:23:36.321,False,False,True,True,NaN,NaN,486.0,487.0,None,None,S1A_WV_SLC__1SSV_20220313T162301_20220313T1631...,S1A_WV_OCN__2SSV_20220313T162301_20220313T1631...
8,2022-03-26 02:22:48.160,2022-03-26 02:22:48.456,False,False,False,False,NaN,NaN,NaN,NaN,None,None,None,None
9,2022-04-09 13:08:05.246,2022-04-09 13:08:05.397,True,True,True,False,32.0,24.0,27.0,NaN,S1A_IW_RAW__0SDV_20220409T130800_20220409T1308...,S1A_IW_GRDH_1SDV_20220409T130803_20220409T1308...,S1A_IW_SLC__1SDV_20220409T130737_20220409T1308...,None


## Save acquisition results to csv file

In [9]:
start_date = acquisitions.iloc[0].start.isoformat()
csv_filename = f"{start_date}.csv"
acquisitions.to_csv(csv_filename)
print(f"CSV file saved to {csv_filename}")

CSV file saved to 2022-02-07T05:54:08.340000.csv
